Zomato restaurants data

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get("https://www.zomato.com/gwalior/restaurants")
time.sleep(3)

# SCROLL PAGE TO LOAD ALL CARDS
last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(15):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# GET ALL CARDS
cards = driver.find_elements(By.XPATH, "//a[contains(@href,'/info')]")
print("CARDS FOUND:", len(cards))

restaurants = []

for card in cards:
    # SCROLL CARD INTO VIEW — THIS RENDERS THE TEXT
    driver.execute_script("arguments[0].scrollIntoView(true);", card)
    time.sleep(0.8)   # small wait for React to render text

    link = card.get_attribute("href")

    try:
        name = card.find_element(By.TAG_NAME, "h4").text
    except:
        name = None

    # pick p tags
    ps = card.find_elements(By.TAG_NAME, "p")

    cuisine = ps[0].text if len(ps) > 0 else None
    price = ps[1].text if len(ps) > 1 else None

    restaurants.append({
        "name": name,
        "cuisine": cuisine,
        "price": price,
        "link": link
    })

df = pd.DataFrame(restaurants)
df.to_csv("raw_data/zomato_full_data_gwalior.csv", index=False)

print("SCRAPED:", len(df))
driver.quit()


Extra details

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

df = pd.read_csv("zomato_clean_final.csv")  # your file with restaurant links

details = []

for i, row in df.iterrows():
    link = row['link']
    print("SCRAPING DETAILS:", link)

    try:
        driver.get(link)
        time.sleep(2)

        # Name
        try:
            name = driver.find_element(By.TAG_NAME, "h1").text
        except:
            name = None

        # Rating
        try:
            rating = driver.find_element(By.CSS_SELECTOR, ".sc-1q7bklc-1").text
        except:
            rating = None

        # Cuisine list
        try:
            cuisine_tags = driver.find_elements(By.CSS_SELECTOR, ".sc-1hez2tp-0")
            cuisines = ", ".join([c.text for c in cuisine_tags[:3]])
        except:
            cuisines = None

        # Address
        try:
            address = driver.find_element(By.CSS_SELECTOR, ".sc-1hez2tp-0.sc-bku4xd-1").text
        except:
            address = None

        # Cost for two
        try:
            cost = driver.find_element(By.XPATH, "//p[contains(text(),'for two')]").text
        except:
            cost = None

        # Timings
        try:
            timings = driver.find_element(By.CSS_SELECTOR, ".sc-kZmsYB").text
        except:
            timings = None

        details.append({
            "name": name,
            "rating": rating,
            "cuisines": cuisines,
            "address": address,
            "cost_for_two": cost,
            "timings": timings,
            "link": link
        })

    except Exception as e:
        print("ERROR:", e)
        continue

pd.DataFrame(details).to_csv("raw_data/zomato_details.csv", index=False)
print("DONE — RESTAURANT DETAILS SAVED!")
driver.quit()
